<a href="https://colab.research.google.com/github/Grecia329/EcuacionesDiferencialesParciales-/blob/main/Metodo_Jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Método de Jacobi

Introducción:

El **método de Jacobi** es un procedimiento iterativo para resolver sistemas de ecuaciones lineales de la forma $A x = b$.

Primero, la matriz \(A\) se descompone como:
$$
A = D + (L+U),
$$
donde $D$ es la matriz diagonal de $A$, $L$ su parte estrictamente triangular inferior y $U$ la parte estrictamente triangular superior.

A partir de esta descomposición, se define la iteración:
$$
x^{(k+1)} = D^{-1}\bigl(b - (L+U)x^{(k)}\bigr)
= -D^{-1}(L+U)x^{(k)} + D^{-1}b.
$$
Esta expresión se puede escribir como:
$$
x^{(k+1)} = B x^{(k)} + c, \quad \text{con } B = -D^{-1}(L+U), \quad c = D^{-1}b.
$$
El proceso se repite hasta que se cumpla un **criterio de paro**, normalmente cuando la norma del cambio entre iteraciones $\|x^{(k+1)}-x^{(k)}\|\$ es menor que una tolerancia predefinida.


In [27]:
# =============================================================================
#  Método de Jacobi — Desarrollo del código + implementación del código con el ejercicio del problemario
#  Grecia Ávila — Colab/Jupyter
#
#  Qué hace:
#   - Implementa Jacobi con documentación y comentarios
#   - Verifica dominancia diagonal (aviso)
#   - (Opcional) estima radio espectral de la matriz de iteración B
#   - Corre el ejercicio dado (A y b) y muestra una TABLA por iteración:
#       k, x1..xn, ||Δx||_inf, ||r||_inf
# =============================================================================

import numpy as np
import pandas as pd
from IPython.display import display

def jacobi(A, b, x0=None, tol=1e-8, maxit=100, return_history=True, check_convergence=True):
    """
    Método de Jacobi para resolver A x = b.

    Parámetros
    ----------
    A : array (n,n)
        Matriz del sistema.
    b : array (n,)
        Vector de términos independientes.
    x0 : array (n,), opcional
        Aproximación inicial. Si no se da, usa vector cero.
    tol : float
        Tolerancia para los criterios de paro:
        - ||x^{k+1} - x^{k}||_inf <= tol  (cambio entre iteraciones)
        - ||r||_inf <= tol, con r = b - A x^{k+1} (residuo)
    maxit : int
        Máximo de iteraciones.
    return_history : bool
        Si True, devuelve también un DataFrame con la tabla de iteraciones.
    check_convergence : bool
        Si True, imprime chequeos útiles (dominancia diagonal, radio espectral).

    Returns
    -------
    x : array (n,)
        Aproximación final.
    k : int
        Iteraciones realizadas.
    history : pd.DataFrame (opcional)
        Tabla con columnas: k, x1..xn, ||Δx||_inf, ||r||_inf.
    """
    # --- Preparación y validaciones básicas ---
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1)
    n = A.shape[0]
    assert A.shape == (n, n), "A debe ser cuadrada"
    assert b.shape == (n,), "b debe ser compatible con A"
    if np.any(np.isclose(np.diag(A), 0.0)):
        raise ValueError("Hay ceros (o casi ceros) en la diagonal de A; Jacobi no es aplicable sin reordenar.")

    # --- Chequeos de convergencia (suficientes, no necesarios) ---
    if check_convergence:
        diag_abs = np.abs(np.diag(A))
        sum_no_diag = np.sum(np.abs(A), axis=1) - diag_abs
        dd = np.all(diag_abs > sum_no_diag)  # estricta
        if dd:
            print("A es estrictamente diagonalmente dominante por renglones (sugiere convergencia).")
        else:
            print("A NO es estrictamente diagonalmente dominante. Jacobi podría converger o no.")
        # Estimar radio espectral de B cuando es pequeño (criterio suficiente: ρ(B)<1)
        try:
            D_inv = np.diag(1.0 / np.diag(A))
            B = - D_inv @ (A - np.diag(np.diag(A)))
            rho = max(abs(np.linalg.eigvals(B)))
            print(f"ℹ Radio espectral estimado ρ(B) ≈ {rho:.6f} (si ρ<1, Jacobi converge).")
        except Exception:
            pass

    # --- Descomposición A = D + (L+U) ---
    D = np.diag(np.diag(A))
    LU = A - D
    D_inv = np.diag(1.0 / np.diag(D))

    # Matriz de iteración y término independiente de la iteración
    B = - D_inv @ LU
    c = D_inv @ b

    # Aproximación inicial
    x_old = np.zeros(n) if x0 is None else np.array(x0, dtype=float).reshape(-1)

    # --- Bucle iterativo de Jacobi ---
    history_rows = []
    for k in range(1, maxit + 1):
        x_new = B @ x_old + c  # fórmula de Jacobi
        err = np.linalg.norm(x_new - x_old, ord=np.inf)      # ||Δx||_inf
        res = np.linalg.norm(b - A @ x_new, ord=np.inf)      # ||r||_inf

        # Guardamos fila para la tabla
        row = {"k": k, **{f"x{i+1}": x_new[i] for i in range(n)}, "||Δx||_inf": err, "||r||_inf": res}
        history_rows.append(row)

        # Criterios de paro
        if err <= tol or res <= tol:
            break

        x_old = x_new

    x = x_new
    if return_history:
        df = pd.DataFrame(history_rows)
        return x, k, df
    else:
        return x, k

# =============================================================================
#  EJERCICIO DEL PROBLEMARIO 1
# =============================================================================
A = np.array([
    [ 4, -1, -1,  0],
    [-1,  4,  0, -1],
    [-1,  0,  4, -1],
    [ 0, -1, -1,  4]
], dtype=float)

b = np.array([0, 2/3, 8/9, 14/9], dtype=float)

# Parámetros de corrida
x0   = np.zeros(4)
tol  = 1e-12         # tolerancia estricta
maxit= 500

# Ejecutar Jacobi
x, k, tabla = jacobi(A, b, x0=x0, tol=tol, maxit=maxit, return_history=True)

# --- Impresión de resultados ---
print("\nSolución aproximada:")
for i, xi in enumerate(x, start=1):
    print(f"x{i} ≈ {xi:.10f}")
print(f"Iteraciones: {k}")

print("\nTabla de iteraciones:")
with pd.option_context("display.float_format", "{:,.6e}".format):
    display(tabla)  # en Colab/Jupyter se muestra como tabla bonita

# --- Guardados útiles para el repositorio ---
tabla.to_csv("jacobi_iteraciones.csv", index=False)
np.savetxt("solucion_jacobi.txt", x, fmt="%.12f")



A es estrictamente diagonalmente dominante por renglones (sugiere convergencia).
ℹ Radio espectral estimado ρ(B) ≈ 0.500000 (si ρ<1, Jacobi converge).

Solución aproximada:
x1 ≈ 0.1944444444
x2 ≈ 0.3611111111
x3 ≈ 0.4166666667
x4 ≈ 0.5833333333
Iteraciones: 39

Tabla de iteraciones:


,k,x1,x2,x3,x4,||Δx||_inf,||r||_inf
0,1,0.000000e+00,1.666667e-01,2.222222e-01,3.888889e-01,3.888889e-01,3.888889e-01
1,2,9.722222e-02,2.638889e-01,3.194444e-01,4.861111e-01,9.722222e-02,1.944444e-01
2,3,1.458333e-01,3.125000e-01,3.680556e-01,5.347222e-01,4.861111e-02,9.722222e-02
3,4,1.701389e-01,3.368056e-01,3.923611e-01,5.590278e-01,2.430556e-02,4.861111e-02
4,5,1.822917e-01,3.489583e-01,4.045139e-01,5.711806e-01,1.215278e-02,2.430556e-02
5,6,1.883681e-01,3.550347e-01,4.105903e-01,5.772569e-01,6.076389e-03,1.215278e-02
6,7,1.914062e-01,3.580729e-01,4.136285e-01,5.802951e-01,3.038194e-03,6.076389e-03
7,8,1.929253e-01,3.595920e-01,4.151476e-01,5.818142e-01,1.519097e-03,3.038194e-03
8,9,1.936849e-01,3.603516e-01,4.159071e-01,5.825738e-01,7.595486e-04,1.519097e-03
9,10,1.940647e-01,3.607313e-01,4.162869e-01,5.829536e-01,3.797743e-04,7.595486e-04


**Salida del Código:**

Podemos utilizar el método de Jacobi en base a la definición de radio espectral para convergencia.

La solución aproximada:
$$
U_{11} = x_1 ≈ 0.1944444 \quad
U_{12} = x_2 ≈ 0.3611111 \quad
U_{21} = x_3 ≈ 0.4166666 \quad
U_{22} = x_4 ≈ 0.5833333 \quad
$$

Cuantas iteraciones se realizaron.

Finalmente se puede observar la tabla con las diversas iteraciones utilizando el método.